In [3]:
import numpy as np
from scipy.integrate import odeint

# Define the global parameter dictionary
Par = {
    'Tl': 0  # You can set the actual value for Tl here
}

def DC_motor_Obs(X, t, Par):
    # Extract state variables
    x = X[:3]
    xh = X[3:]

    # Define matrices
    A = np.array([
        [0, 1, 0],
        [0, 0, 4.438],
        [0, -12, -24]
    ])
    
    B = np.array([
        [0, 0],
        [0, -7.396],
        [20, 0]
    ])
    
    C = np.array([1, 0, 0])

    Ah = np.array([
        [0, 1, 0, 0],
        [0, 0, 4.438, -7.396],
        [0, -12, -24, 0],
        [0, 0, 0, 0]
    ])
    
    Bh = np.array([0, 0, 20, 0])
    Ch = np.array([1, 0, 0, 0])

    G = np.array([0, 234.7440, -936.9136, -27.6050])
    
    # Step disturbance
    Tl = Par['Tl']
    v = 0
    u = np.array([v, Tl])
    
    # Calculate the real system model
    xp = np.dot(A, x) + np.dot(B, u)
    y = np.dot(C, x)
    
    # Calculate the observer model
    xhp = np.dot(Ah, xh) + np.dot(Bh, v) + np.dot(G.reshape(-1, 1), (y - np.dot(Ch, xh)).reshape(1, -1)).flatten()
    
    # Augment the real and estimated states
    Xp = np.concatenate((xp, xhp))
    return Xp

# Example usage with initial conditions and time span
X0 = np.zeros(7)  # Initial conditions for the state variables
t = np.linspace(0, 10, 100)  # Time span for the simulation

# Integrate the system of differential equations
X = odeint(DC_motor_Obs, X0, t, args=(Par,))

# Extract the real and estimated states
x_real = X[:, :3]
x_hat = X[:, 3:]
